In [2]:
import utils

import re
from collections import Counter

## Day 5: Print Queue

[#](https://adventofcode.com/2024/day/5) We have a list of page ordering rules and the pages to produce in each update.

Page order rules: `47|53` means that if an update includes both these numbers, 47 has to be printed first.
Page numbers in an update: `75,47,61,53,29` defines the order of pages printed.

So we need a function:

1. Check if an update passes all rules
2. Return middle page number of that update

In [112]:
sample_input: str = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""

puzzle_input = utils.get_input(5, splitlines=False)

In [137]:
def get_nums(line: str) -> list:
    """takes a string, returns a list of numbers found
    '47|53' -> (47, 53)"""
    return [int(x) for x in re.findall(r"\d+", line)]

In [138]:
def parse_input(input_str=sample_input, debug: bool = False):

    rules_str, update_str = input_str.strip().split("\n\n")

    rules, updates = [], []

    for i, line in enumerate(rules_str.splitlines()):
        numbers = get_nums(line)
        if debug and i < 3:
            print(f"Line {i}: '{line}' | Extracted: {numbers}")
        rules.append(numbers)

    for i, line in enumerate(update_str.splitlines()):
        numbers = get_nums(line)
        if debug and i < 3:
            print(f"Line {i}: '{line:15}' | Extracted: {numbers}")
        updates.append(numbers)

    return rules, updates


rules, updates = parse_input(sample_input, True)

Line 0: '47|53' | Extracted: [47, 53]
Line 1: '97|13' | Extracted: [97, 13]
Line 2: '97|61' | Extracted: [97, 61]
Line 0: '75,47,61,53,29 ' | Extracted: [75, 47, 61, 53, 29]
Line 1: '97,61,53,29,13 ' | Extracted: [97, 61, 53, 29, 13]
Line 2: '75,29,13       ' | Extracted: [75, 29, 13]


In [139]:
def check_update(update=updates[0], rules=rules, debug=False) -> bool:
    """returns true if an update meets all the rules"""
    for rule in rules:
        first, second = rule
        if (first in update) and (second in update):
            if not update.index(second) > update.index(first):
                if debug:
                    print(f"{rule} violated in {update=}")
                return False
    return True


check_update(updates[4], debug=True)

[29, 13] violated in update=[61, 13, 29]


False

In [140]:
def get_middle_num(lst) -> int:
    """returns the middle number in a odd list, middle+1 in a even list"""
    assert len(lst) % 2 != 0  # whats middle num of a even len lst?
    return lst[len(lst) // 2]

In [141]:
def solve(inp: str = sample_input, debug: bool = False):
    rules, updates = parse_input(inp)

    nums = []
    for update in updates:
        if check_update(update, rules, debug):
            nums.append(get_middle_num(update))

    ans = sum(nums)
    if debug:
        print(f"{ans}, {nums=}")

    return {"result": ans}


assert solve(sample_input, True)["result"] == 143  # sample ans check

results = solve(puzzle_input, debug=False)
print(f"\nPart 1: {results["result"]}")  # 5643 is too high

[97, 75] violated in update=[75, 97, 47, 61, 53]
[29, 13] violated in update=[61, 13, 29]
[29, 13] violated in update=[97, 13, 75, 29, 47]
143, nums=[61, 53, 29]

Part 1: 4872


## Part 2

Get all the incorrectly ordered updates and use the rules to put them in the right order.

This one was tricky - as part two is recursive. Every time you apply a rule to fix an update, you effectively have a new update, to which you need to apply all the rules again.

I was thinking of how to do this... until after some minutes I remembered recursion. We have a default case - once the last rule is appled, we can just return the update as it was passed in, which makes the recursive function easy to think about:

In [163]:
def fix_update(update, rules, debug=False):
    """recursively fixes updates"""
    for rule in rules:
        first, second = rule
        if (first in update) and (second in update):
            i = update.index(first)
            j = update.index(second)

            if i > j:  # need to swap
                update[i], update[j] = (
                    update[j],
                    update[i],
                )
                # we have a new update, need to check it for all the rules
                return fix_update(update, rules, debug)

    return update


fix_update(update, rules)

[97, 75, 47, 29, 13]

In [164]:
def solve_2(inp: str = sample_input, debug: bool = False):
    rules, updates = parse_input(inp)

    nums = []

    for update in (
        update for update in updates if not check_update(update, rules, debug)
    ):
        if debug:
            print(f"{update}")
        lst = fix_update(update, rules)
        if debug:
            print(f"{lst=}")
        nums.append(get_middle_num(lst))

    ans = sum(nums)
    if debug:
        print(f"{ans}, {nums=}")
    return {"result": ans}


solve_2(sample_input, debug=True)
assert solve_2(sample_input)["result"] == 123  # p2 sample input answer

results = solve_2(puzzle_input, debug=False)
print(f"\nPart 2: {results["result"]}")

[97, 75] violated in update=[75, 97, 47, 61, 53]
[75, 97, 47, 61, 53]
lst=[97, 75, 47, 61, 53]
[29, 13] violated in update=[61, 13, 29]
[61, 13, 29]
lst=[61, 29, 13]
[29, 13] violated in update=[97, 13, 75, 29, 47]
[97, 13, 75, 29, 47]
lst=[97, 75, 47, 29, 13]
123, nums=[47, 29, 47]

Part 2: 5564
